### Reconocimiento de enfermedades previas en informes de urgencias y su mapeado con el correspondiente código CIE10

Vamos a utilizar nuestro modelo aplicándolo a informes de urgencias reales

In [1]:
import json
from glob import glob
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
from medspacy.target_matcher import TargetMatcher, TargetRule
from medspacy.context import ConTextRule, ConTextComponent

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

D:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cargamos el modelo entrenado

In [3]:
nlp = spacy.load("./models/model_antecedentes_ner_es_md")

Añadimos al modelo el componente que añade los códigos CEI10 a los antecedentes que encuentre

In [4]:
target_matcher = TargetMatcher(nlp)
Span.set_extension("CIE10", default="", force=True)
rules = TargetRule.from_json("./data/cie10_rules_mspacy.jsonl")
target_matcher.add(rules)
nlp.add_pipe(target_matcher)
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner', 'target_matcher']

En los textos clínicos muchas veces hay menciones a conceptos que en este momento el paciente no padece.
Eliminamos los antecedentes de negación con las reglas presentes en el archivo context_rules.jsonl

In [5]:
context = ConTextComponent(nlp, rules=None)
context_rules = ConTextRule.from_json("./data/context_rules.jsonl")
context.add(context_rules)
nlp.add_pipe(context)
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner', 'target_matcher', 'context']

Vemos que tal funciona el modelo con nuevos informes de urgencias

In [6]:
# directorio dónde están cada uno de los txt que contienen un informe de urgencias
inf_nuevos = glob("./data/informes/*.txt")

# generamos un archivo json que contine el nombre de cada archivo con los antedentes y sus códigos encontrados por el modelo
with open("data/antecedentes_informes_urgencias.jsonl", "w", encoding="utf-8" ) as outfile: 
    archivo_antec=[]
    for f in inf_nuevos:
        with open(f, "r", encoding="utf-8") as infile:
            nombre_archivo = f
            nombre_archivo = nombre_archivo.replace("./data/informes\\", "")
            nombre_archivo = nombre_archivo.replace(".txt", "")
            text = infile.read()            
            #print (text)
            nuevo_informe = nlp(text)
            ante_encontrados = nombre_archivo, [[ent._.CIE10, ent.text] for ent in nuevo_informe.ents if ent._.is_negated == False]
            archivo_antec.append(ante_encontrados)
            json.dump(ante_encontrados, outfile, indent=1)

In [7]:
antec_encontrados = [[archivo, *antec] for archivo, lista in archivo_antec for antec in lista]

In [11]:
df = pd.DataFrame(antec_encontrados)
df.columns = ["Archivo", "CEI10", "Antecedente"]
# lo guardamos en un archivo excel
df.to_excel('data/Antecedentes_CEI10_Encontrados_Informes_Urgencias_Con_Negacion.xlsx')

In [12]:
print (f"Número de antecedentes encontrados: {len(antec_encontrados)}\n")
df

Número de antecedentes encontrados: 15



,Archivo,CEI10,Antecedente
0,inf2,F17.210,Fumadora
1,inf2,C07,Carcinoma
2,inf2,C07,Carcinoma
3,inf2,C50.9,CARCINOMA DUCTAL INFILTRANTE
4,inf2,I82.409,TVP
5,inf3,J45.909,Asma extrínseca
6,inf3,Z87.891,Exfumadora
7,inf4,I10,HTA
8,inf4,E11.9,DM tipo 2
9,inf4,I25.9,Cardiopatía isquémica


Este es el archivo resultante de aplicar el modelo a los informes de urgencias [Antecedentes_CEI10_Encontrados_Informes_Urgencias_Con_Negacion.xlsx](data/Antecedentes_CEI10_Encontrados_Informes_Urgencias_Con_Negacion.xlsx)